In [23]:
from sklearn.metrics import mean_squared_error


In [24]:
import requests
import json

def verify_predictions(y_pred):
    
    url = "https://q7wmplg8u6.execute-api.ap-northeast-1.amazonaws.com/dev"
    
    payload = {
        "submission": y_pred
    }
    response = requests.request("POST", url,data=json.dumps(payload),headers = {'content-type': 'application/json'})
    
    return response.text

In [25]:
# imports
import numpy as np
import pandas as pd

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# loading the datasets
df = pd.read_csv('/content/drive/MyDrive/wni/Japan_cities_rainfall.csv')
df_to_predict = pd.read_csv('/content/drive/MyDrive/wni/rainfall_to_predict.csv')

In [28]:
df.head()

,date_time,place,longitude,latitude,rainfall
0,20200918_080000,Naha,127.686667,26.206667,0.0
1,20200918_080000,Fukuoka,130.375000,33.581667,0.0
2,20200918_080000,Sendai,140.896667,38.261667,0.0
3,20200918_080000,Osaka,135.518333,34.681667,0.0
4,20200918_080000,Nigata,139.018333,37.893333,3.0


In [29]:
df_to_predict.head()

,date_time,place,longitude,latitude
0,20200919_013000,Naha,127.686667,26.206667
1,20200919_223000,Nigata,139.018333,37.893333
2,20200919_233000,Sendai,140.896667,38.261667
3,20200920_030000,Fukuoka,130.375000,33.581667
4,20200920_130000,Naha,127.686667,26.206667


In [30]:
def breakdown_month(a):
  if(a[4:6]=='9'):
      return 0
  elif(a[4:6]=='10'):
      return 1
  else:
      return 2
def breakdown_date(a):
  return int(a[6:8])
def breakdown_time(a):
  if(int(a[9:11])>=0 and int(a[9:11])<6):
    return 1
  elif(int(a[9:11])>=6 and int(a[9:11])<12):
    return 2
  elif(int(a[9:11])>=12 and int(a[9:11])<18):
    return 3
  elif(int(a[9:11])>=18 and int(a[9:11])<24):
    return 4
  

In [31]:
city_mapping = {" Naha": 1, " Fukuoka": 2, " Sendai": 3, " Osaka": 4, " Nigata": 5, " Tokyo": 6}
datasets=[df,df_to_predict]
cols=['date','month','time','city','rainfall']
for data in datasets:
  data['city']=data['place'].map(city_mapping)
  data['month']=data['date_time'].apply(breakdown_month)
  data['date']=data['date_time'].apply(breakdown_date)
  data['time']=data['date_time'].apply(breakdown_time)
  data.drop(['place','longitude','latitude','date_time'],axis=1,inplace=True)
df=df[cols]
df_to_predict=df_to_predict[cols[:-1]]

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor  
Y=df['rainfall']
X=df.drop('rainfall',axis=1)
gbr_model=GradientBoostingRegressor()
gbr_model.fit(X, np.ravel(Y))
pred=gbr_model.predict(df_to_predict)
pred=pred.tolist()
result_rmse = verify_predictions(pred)
print(result_rmse)
print(df_to_predict[:10])
print(pred[:10])

{"result":0.7850236515182961,"type":"rmse"}

   date  month  time  city
0    19      2     1     1
1    19      2     4     5
2    19      2     4     3
3    20      2     1     2
4    20      2     3     1
5    20      2     4     1
6    20      2     4     5
7    21      2     2     5
8    21      2     3     3
9    21      2     4     5
[0.2282296131461955, 0.10497764087733645, 0.021350371578241763, 0.05753104092730099, 0.13906219945823953, 0.1336191478211305, 0.10497764087733645, 0.04139197265440487, 0.06135497316948136, 0.0631900478272158]
